In [ ]:
# Parámetro para generar el modelo del taggeador
data = True
%store data

In [ ]:
%%time
%run ./carga_diputados.ipynb

In [ ]:
%%time
%run ./carga_dialogos.ipynb

In [1]:
import neo4j_connector as nc
import pandas as pd
from tqdm import tqdm
import pickle

In [12]:
graph = nc.generate_graph()

In [13]:
grupos_parlamentarios = list(graph.run('MATCH (d:Diputado) RETURN DISTINCT(d.grupo) AS grupo ORDER BY grupo ASC').to_data_frame()['grupo'])

### OBTENEMOS LISTA DE RELACIONES

In [14]:
query = 'MATCH ()-[relation]-() RETURN DISTINCT(relation.pleno) as sesion order by sesion asc'
sesiones = graph.run(query).to_data_frame()

In [15]:
def lista_palabras(grupo, plenos):
    query = "MATCH (d:Diputado)-[rel]-(p:Palabra) "
    query = query + "WHERE d.grupo = '" + grupo +"' "
    query = query + "AND rel.pleno IN " + str(plenos) + " "
    query = query + "RETURN rel.pleno as pleno, d.grupo as grupo, p.palabra as palabra, SUM(rel.veces) as veces "
    query = query + "ORDER BY veces DESC, palabra ASC LIMIT 15"
    return graph.run(query).to_data_frame()

In [16]:
def lista_palabras_hist(grupo, plenos):
    query = "MATCH (d:Diputado)-[rel]-(p:Palabra) "
    query = query + "WHERE d.grupo = '" + grupo +"' "
    query = query + "AND rel.pleno IN " + str(plenos) + " "
    query = query + "RETURN d.grupo as grupo, p.palabra as palabra, SUM(rel.veces) as veces "
    query = query + "ORDER BY veces DESC, palabra ASC "
    return graph.run(query).to_data_frame()

In [17]:
def lista_palabras_mas_dichas_hist(grupo, plenos):
    query = "MATCH (d:Diputado)-[rel]-(p:Palabra) "
    query = query + "WHERE d.grupo = '" + grupo +"' "
    query = query + "AND rel.pleno IN " + str(plenos) + " "
    query = query + "RETURN d.grupo as grupo, p.palabra as palabra, SUM(rel.veces) as veces "
    query = query + "ORDER BY veces DESC, palabra ASC LIMIT 15"
    return graph.run(query).to_data_frame()

In [18]:
historico = []
graficos = []
grafico_hist = []
for ix, sesion in sesiones.iterrows():
    historico.append(sesion['sesion'])
    print(sesion['sesion'])
    ses = []
    ses_hist = []
    for gr in grupos_parlamentarios:
        palabras_mas_dichas = lista_palabras(gr, [sesion['sesion']])
        if palabras_mas_dichas.empty:
            palabras_mas_dichas = pd.DataFrame.from_dict({'pleno': [sesion['sesion']],
                                                          'grupo': [gr],
                                                          'palabra' : ['gobierno'],
                                                          'veces' : [0]
                                                         })
            
        palabras_mas_dichas_hist = lista_palabras_hist(gr, historico)   
        
        if palabras_mas_dichas_hist.empty:
            palabras_mas_dichas_hist = pd.DataFrame.from_dict({'pleno': [sesion['sesion']],
                                                               'grupo': [gr],
                                                               'palabra' : ['gobierno'],
                                                               'veces' : [0]
                                                              })
            
        df = palabras_mas_dichas.merge(palabras_mas_dichas_hist,
                                       how='left', left_on=['grupo', 'palabra'], right_on=['grupo','palabra']
                                      ).rename(columns={'veces_x':'numero_veces', 'veces_y': 'veces_acumuladas'})

        ses.append(df)
        ses_hist.append(lista_palabras_mas_dichas_hist(gr, historico))
    graficos.append([sesion['sesion'], ses])
    grafico_hist.append([sesion['sesion'], ses_hist])

00_INVESTIDURA
2020_02_04
2020_02_11
2020_02_12
2020_02_18
2020_02_19
2020_02_20
2020_02_25
2020_02_26
2020_02_27


In [22]:
import matplotlib.pyplot as plt
%matplotlib inline

In [26]:
output = []
palabras = []
palabras_hist = []
for ind1, grafico_presente in enumerate(graficos):
    for ind2, grupo in enumerate(grafico_presente[1]):
        #cargamos debate, grupo, df palabras mas dichas, df, acumuladas, df totales para poder procesar despues
#        output.append([grupo['pleno'][0], 
#                       grupo['grupo'][0], 
#                       grupo[['palabra', 'numero_veces']], 
#                       grupo[['palabra','veces_acumuladas']], 
#                       grafico_hist[ind1][1][ind2][['palabra', 'veces']]])
        pal = grupo[['palabra', 
               'numero_veces']].join(grupo[['palabra','veces_acumuladas']].set_index('palabra'), 
                                     on='palabra')
        pal['grupo'] = grupo['grupo'][0]
        pal['pleno'] = grupo['pleno'][0]
        pal_hist = grafico_hist[ind1][1][ind2][['palabra', 'veces']].set_index('palabra')
        pal_hist['grupo'] = grupo['grupo'][0]
        pal_hist['pleno'] = grupo['pleno'][0]
        palabras.append(pal)
        palabras_hist.append(pal_hist)

In [27]:
with open('palabras.pickle', 'wb') as handle:
    pickle.dump(pd.concat(palabras), 
                handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('palabras_hist.pickle', 'wb') as handle:
    pickle.dump(pd.concat(palabras_hist), 
                handle, protocol=pickle.HIGHEST_PROTOCOL)